In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://www.yapo.cl/searchresult/bienes-raices-alquiler-apartamentos?regionslug=region-metropolitana'
n_paginas = 424

all_apartments_data = []

for i in range(1, n_paginas+1):
    if i == 1:
        url = 'https://www.yapo.cl/searchresult/bienes-raices-alquiler-apartamentos?regionslug=region-metropolitana'
    else:
        url = f'https://www.yapo.cl/searchresult/bienes-raices-alquiler-apartamentos.{i}?regionslug=region-metropolitana'

    html_text = requests.get(f'{url}').text
    soup = BeautifulSoup(html_text, 'lxml')

    # Find ALL elements that represent an apartment listing
    departamentos = soup.find_all('div', class_='d3-ad-tile__content')

    # Check if any listings were found
    if not departamentos:
      print(f"No apartment listings found on page {i}.")
    else:
      print(f"Scraping page {i}...")
      for departamento in departamentos:
          try:
              # Extraer precio
              precio = departamento.find('div', class_='d3-ad-tile__price')
              precio_text = precio.text.replace(' ', '').replace('\n', '') if precio else "N/A"

              # Extraer comuna
              comuna = departamento.find('svg', class_='d3-icon d3-ad-tile__location-icon')
              comuna_text = comuna.parent.text.replace('\n', '').replace(' ', '') if comuna and comuna.parent else "N/A"

              # Extraer título
              titulo = departamento.find('span', class_='d3-ad-tile__title')
              titulo_text = titulo.text.replace('\n', '').replace(' ', '') if titulo else "N/A"

              # Inicializar variables para m2, dormitorios y baños
              metros_cuadrados = "N/A"
              dormitorios = "N/A"
              banos = "N/A"
              estacionamientos = "N/A"

              # Encontrar todos los elementos de detalles
              detalles_items = departamento.find_all('li', class_='d3-ad-tile__details-item')

              # Iterar sobre los elementos de detalles para extraer la información
              for item in detalles_items:
                  svg_icon = item.find('svg')
                  if svg_icon:
                      use_tag = svg_icon.find('use')
                      if use_tag and 'xlink:href' in use_tag.attrs:
                          icon_type = use_tag['xlink:href']
                          value = item.get_text(strip=True).replace('\n', '').replace(' ', '') # Extraer el texto dentro del li

                          if "#resize" in icon_type:
                              metros_cuadrados = value
                          elif "#bed" in icon_type:
                              dormitorios = value
                          elif "#bath" in icon_type:
                              banos = value
                          elif "#parking" in icon_type:
                              estacionamientos = value

              apartment_data = {
                  'Título': titulo_text,
                  'Comuna': comuna_text,
                  'Precio': precio_text,
                  'Metros Cuadrados': metros_cuadrados,
                  'Dormitorios': dormitorios,
                  'Baños': banos,
                  'Estacionamientos': estacionamientos
              }

              all_apartments_data.append(apartment_data)

          except Exception as e:
              # Handle potential errors if the structure is slightly different for a listing
              print(f"Error extracting data for this listing: {e}")

df = pd.DataFrame(all_apartments_data)
print("\nDataFrame created:")
display(df.head())

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

,Título,Comuna,Precio,Metros Cuadrados,Dormitorios,Baños,Estacionamientos
0,ArriendoDepto1D/50%DCTO/1ERMESGRATISGC/Garantí...,EstaciónCentral,$260.000-4%,N/A,1,1,N/A
1,Depto.2D+1B+E+B/50%DCTO/1ERMESGRATISGC/Garantí...,EstaciónCentral,$340.000-3%Oportunidad,N/A,2,1,1
2,TRESDORMITORIOSUNBAÑOYBODEGAyDOSDORMITORIOSDIS...,EstaciónCentral,$450.000-6%,65m2,3,2,N/A
3,"DEPARTAMENTOThompson4225,EstaciónCentral",EstaciónCentral,$295.000,32m2,1,1,N/A
4,DEPARTAMENTOZañartu980,Ñuñoa,$379.968,37m2,1,1,N/A


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12690 entries, 0 to 12689
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        12690 non-null  int64  
 1   Título            12690 non-null  object 
 2   Comuna            12690 non-null  object 
 3   Precio            12683 non-null  object 
 4   Metros Cuadrados  8486 non-null   object 
 5   Dormitorios       12552 non-null  float64
 6   Baños             12649 non-null  float64
 7   Estacionamientos  3113 non-null   object 
 8   Precio_UF         12690 non-null  bool   
 9   Precio_CLP        12690 non-null  bool   
 10  Precio_Limpio     12683 non-null  float64
 11  Precio_CLP_Final  12683 non-null  float64
dtypes: bool(2), float64(4), int64(1), object(5)
memory usage: 1016.3+ KB


In [ ]:
df.to_csv('departamentos_metropolitana_mayo_2025.csv', index=False)